In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import *
from sklearn.externals import joblib
from difflib import get_close_matches

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
df = pd.read_csv("tmdb_5000_movies.csv")
df1 = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
df.shape

(4803, 20)

In [4]:
df1 = df1.rename(columns = {"movie_id":"id"})
df1.head()

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
result = pd.merge(df,df1[['id','cast','crew']],on = 'id')

In [6]:
result.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
def json_to_list(col):
    for index, row in result.iterrows():
        row_list= []
        x = json.loads(row[col])
        for genre in x:
            row_list.append(genre['name'])
        result.at[index, col] = row_list



In [8]:
cols =['genres', 'keywords', 'production_companies', 'production_countries','spoken_languages','cast']
for col in cols:
    json_to_list(col)

In [9]:
def reduce_list(col):
    for index, row in result.iterrows():
        li = row[col]
        li = li[:5]
        result.at[index, col] = li


In [10]:
cols = ['genres','cast','keywords']
for col in cols:
    reduce_list(col)

In [11]:
result.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",...,2787965087,162.0,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",...,961000000,169.0,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi6]",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]",...,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",Released,A Plan No One Escapes,Spectre,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",...,1084939099,165.0,[English],Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],...,284139100,132.0,[English],Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [12]:
result.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64

In [13]:
result = result[result['vote_average']>6]

In [14]:
len(result.loc[(result['keywords'].str.len() == 0),:])

143

In [15]:
len(result.loc[(result['genres'].str.len() == 0),:])

3

In [16]:
for index,row in result.iterrows():
    crew_list = json.loads(row['crew'])
    for x in crew_list:
        if x['job'] == 'Director':
            result.at[index,'crew'] = x['name']

In [17]:
result.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",...,2787965087,162.0,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",...,961000000,169.0,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi6]",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]",...,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",Released,A Plan No One Escapes,Spectre,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",...,1084939099,165.0,[English],Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],...,284139100,132.0,[English],Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton


In [18]:
result = result[['genres','keywords','overview','popularity','revenue','title','vote_average','vote_count','cast','crew']]

In [19]:
result.rename(columns = {'crew':'director'},inplace = True)

In [20]:
result.dropna()

,genres,keywords,overview,popularity,revenue,title,vote_average,vote_count,cast,director
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",150.437577,2787965087,Avatar,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",139.082615,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi6]",A cryptic message from Bond’s past sends him o...,107.376788,880674609,Spectre,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,112.312950,1084939099,The Dark Knight Rises,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",43.926995,284139100,John Carter,6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton
6,"[Animation, Family]","[hostage, magic, horse, fairy tale, musical]",When the kingdom's most wanted-and most charmi...,48.681969,591794936,Tangled,7.4,3330,"[Zachary Levi, Mandy Moore, Donna Murphy, Ron ...",Nathan Greno
7,"[Action, Adventure, Science Fiction]","[marvel comic, sequel, superhero, based on com...",When Tony Stark tries to jumpstart a dormant p...,134.279229,1405403694,Avengers: Age of Ultron,7.3,6767,"[Robert Downey Jr., Chris Hemsworth, Mark Ruff...",Joss Whedon
8,"[Adventure, Fantasy, Family]","[witch, magic, broom, school of witchcraft, wi...","As Harry begins his sixth year at Hogwarts, he...",98.885637,933959197,Harry Potter and the Half-Blood Prince,7.4,5293,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...",David Yates
11,"[Adventure, Action, Thriller, Crime]","[killing, undercover, secret agent, british se...",Quantum of Solace continues the adventures of ...,107.928811,586090727,Quantum of Solace,6.1,2965,"[Daniel Craig, Olga Kurylenko, Mathieu Amalric...",Marc Forster
12,"[Adventure, Fantasy, Action]","[witch, fortune teller, bondage, exotic island...",Captain Jack Sparrow works his way out of a bl...,145.847379,1065659812,Pirates of the Caribbean: Dead Man's Chest,7.0,5246,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski


In [21]:
cols = ['genres','keywords','cast']
for col in cols:
    result = result[result[col].map(lambda d: len(d)) > 0]

In [22]:
result.describe()
result.isnull().sum()

genres          0
keywords        0
overview        1
popularity      0
revenue         0
title           0
vote_average    0
vote_count      0
cast            0
director        0
dtype: int64

In [23]:
from nltk.corpus import stopwords

def tokeniser(df, col):
    sw = list(set(stopwords.words('english'))) + ['.', ',', '?']
    li = []
    for sentt in df[col]:
        words = word_tokenize(str(sentt).lower())
        words = [w for w in words if w not in sw]
        final = ''
        for x in words:
            final = final + " " + x
        li.append(final)
    df[col] = li

tokeniser(result,'overview')

In [24]:
result['title'] = result['title'].apply(lambda x:x.lower())

In [25]:
result = result.drop('overview',axis=1)

In [26]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]    

In [27]:
features = ['cast', 'keywords', 'genres']
for feature in features:
    result[feature] = result[feature].apply(clean_data)

In [28]:
result.head()

,genres,keywords,popularity,revenue,title,vote_average,vote_count,cast,director
0,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...",150.437577,2787965087,avatar,7.2,11800,"[samworthington, zoesaldana, sigourneyweaver, ...",James Cameron
1,"[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...",139.082615,961000000,pirates of the caribbean: at world's end,6.9,4500,"[johnnydepp, orlandobloom, keiraknightley, ste...",Gore Verbinski
2,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6]",107.376788,880674609,spectre,6.3,4466,"[danielcraig, christophwaltz, léaseydoux, ralp...",Sam Mendes
3,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...",112.312950,1084939099,the dark knight rises,7.6,9106,"[christianbale, michaelcaine, garyoldman, anne...",Christopher Nolan
4,"[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...",43.926995,284139100,john carter,6.1,2124,"[taylorkitsch, lynncollins, samanthamorton, wi...",Andrew Stanton


In [29]:
result['director'] = result['director'].str.replace(' ','')
result['director'] = result['director'].str.lower()
result.head()

,genres,keywords,popularity,revenue,title,vote_average,vote_count,cast,director
0,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...",150.437577,2787965087,avatar,7.2,11800,"[samworthington, zoesaldana, sigourneyweaver, ...",jamescameron
1,"[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...",139.082615,961000000,pirates of the caribbean: at world's end,6.9,4500,"[johnnydepp, orlandobloom, keiraknightley, ste...",goreverbinski
2,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6]",107.376788,880674609,spectre,6.3,4466,"[danielcraig, christophwaltz, léaseydoux, ralp...",sammendes
3,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...",112.312950,1084939099,the dark knight rises,7.6,9106,"[christianbale, michaelcaine, garyoldman, anne...",christophernolan
4,"[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...",43.926995,284139100,john carter,6.1,2124,"[taylorkitsch, lynncollins, samanthamorton, wi...",andrewstanton


In [30]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['genres'])
result['soup'] = result.apply(create_soup, axis=1)
result['soup'] = result[['soup','director']].apply(lambda x:' '.join(x), axis=1)    

In [31]:
result['soup'][3]

'dccomics crimefighter terrorist secretidentity burglar christianbale michaelcaine garyoldman annehathaway tomhardy action crime drama thriller christophernolan'

In [32]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(result['soup'])
joblib.dump(count_matrix,"count_model_movie.pkl")

['count_model_movie.pkl']

In [33]:
cos = cosine_similarity(count_matrix,count_matrix)

In [34]:
result = result.reset_index()
indices = pd.Series(result.index, index=result['title'])

In [35]:
def get_recommendations(title, cos=cos):
    idx = indices[title]
    cos_scores = list(enumerate(cos[idx]))
    cos_scores = sorted(cos_scores, key=lambda x: x[1], reverse=True)
    cos_scores = cos_scores[1:11]  
    movie_indices = [i[0] for i in cos_scores]
    return result['title'].iloc[movie_indices]

In [36]:
get_recommendations('avatar')

209                       the fifth element
42                         star trek beyond
35                  star trek into darkness
64                  guardians of the galaxy
1781        star wars: clone wars: volume 1
10                             man of steel
34               x-men: days of future past
58      captain america: the winter soldier
156                           the wolverine
362                               the abyss
Name: title, dtype: object

In [37]:
def close_matches(patterns,word):
    print(get_close_matches(word,patterns,n=5,cutoff=0.5))
close_matches(result['title'],'the')

['three', 'ted', 'her', 'the vow', 'the fog']


In [38]:
result.to_csv('tmdb.csv',index = False, header = True)